In [13]:
import numpy as np 
import cv2 as cv
import os

rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

dataSet = r'C:\Proyectos\IA\Emocione\Emociones'
faces = os.listdir(dataSet)
print(faces)

['feliz', 'sorprendido', 'triste']


In [ ]:
def escala(imx, escala):
    width = int(imx.shape[1] * escala / 100)
    height = int(imx.shape[0] * escala / 100)
    size = (width, height)
    im = cv.resize(imx, size, interpolation = cv.INTER_AREA)
    return im

In [ ]:
cap = cv.VideoCapture(0)
i = 0 
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    for(x, y, w, h) in rostros:
        #frame = cv.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        frame2 = frame[y:y+h, x:x+w]
        #cv.imshow('rostros2', frame2)
        frame2 = cv.resize(frame2, (100,100), interpolation = cv.INTER_AREA)
        cv.imwrite('C:/Proyectos/caras/Emocione/Emociones/Feliz/SinaFeliz'+str(i)+'.png', frame2)
                    
    cv.imshow('rostros', frame)
    i=i+1
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
## LBPH

In [ ]:
dataSet = 'C:/Proyectos/caras/Emocione/Emociones'
faces  = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
target_size = (100, 100)
for face in faces:
    facePath = dataSet+'/'+face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0))
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

LBPHFace = cv.face.LBPHFaceRecognizer_create()
LBPHFace.train(facesData, np.array(labels))
LBPHFace.write('emocionesLBPH.xml')

In [16]:
faces  = ['feliz', 'sorprendido', 'triste']
LBPHFace = cv.face.LBPHFaceRecognizer_create()
LBPHFace.read('emocionesLBPH.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier(cv.data.haarcascades +'haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = LBPHFace.predict(frame2)
        if result[1] > 93:
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Neutral',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
    
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()